In [3]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from datetime import timedelta

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor

In [4]:
urbanbus_data = pd.read_csv('../urbanbus_data/SER_0b91_start_aggregated.csv')

df = urbanbus_data.groupby(["Ride_start_datetime", "Bus_Service_Number", "Direction", "Boarding_stop_stn", "Alighting_stop_stn"], as_index=False)["Passenger_Count"].sum()
df['Ride_start_datetime'] = pd.to_datetime(df['Ride_start_datetime'], errors='coerce')
df = df.sort_values('Ride_start_datetime').reset_index(drop=True)

print(f"Total records: {len(df):,}")
print(f"Date range: {df['Ride_start_datetime'].min()} to {df['Ride_start_datetime'].max()}\n")

Total records: 600,672
Date range: 2017-10-01 00:00:00 to 2018-03-31 23:45:00



In [5]:
df.head()

,Ride_start_datetime,Bus_Service_Number,Direction,Boarding_stop_stn,Alighting_stop_stn,Passenger_Count
0,2017-10-01 00:00:00,SER_0b91,Start,AAW_1,APN_1,1
1,2017-10-01 00:00:00,SER_0b91,Start,AAW_1,CNA_1,1
2,2017-10-01 00:15:00,SER_0b91,Start,APN_1,AHP_1,1
3,2017-10-01 05:45:00,SER_0b91,Start,IH_2,EQI_1,4
4,2017-10-01 05:45:00,SER_0b91,Start,IH_2,EHI_1,1


In [6]:
# Datetime features
df['hour'] = df['Ride_start_datetime'].dt.hour
df['minute'] = df['Ride_start_datetime'].dt.minute
df['day'] = df['Ride_start_datetime'].dt.day
df['dayofweek'] = df['Ride_start_datetime'].dt.dayofweek
df['month'] = df['Ride_start_datetime'].dt.month
df['year'] = df['Ride_start_datetime'].dt.year
df['week_of_year'] = df['Ride_start_datetime'].dt.isocalendar().week

# Cyclic encoding
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# Peak hour flags
peak_hours = df.groupby('hour')['Passenger_Count'].sum().nlargest(2).index.tolist()
df['is_peak_hour'] = df['hour'].isin(peak_hours).astype(int)
print(peak_hours)

# Weekend and holiday flag
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
china_holidays = holidays.country_holidays('CN')
df['is_holiday'] = df['Ride_start_datetime'].dt.date.isin(china_holidays).astype(int)

# Route identifier
df['route'] = df['Boarding_stop_stn'] + '_to_' + df['Alighting_stop_stn']
df = df.sort_values(['route', 'Ride_start_datetime']).reset_index(drop=True)

# Lag Features
for lag in [1, 2, 3, 4, 8, 12, 24]:
    df[f'lag_{lag}'] = df.groupby('route')['Passenger_Count'].shift(lag)

for window in [4, 8, 12, 24]:
    df[f'rolling_mean_{window}'] = (
        df.groupby('route')['Passenger_Count']
        .shift(1)
        .rolling(window=window)
        .mean()
    )
    df[f'rolling_std_{window}'] = (
        df.groupby('route')['Passenger_Count']
        .shift(1)
        .rolling(window=window)
        .std()
    )


lag_roll_cols = [col for col in df.columns if col.startswith(('lag_', 'rolling_'))]
df = df.dropna(subset=lag_roll_cols).reset_index(drop=True)

[7, 18]


In [7]:
max_date = df['Ride_start_datetime'].max()
cutoff_date = max_date - timedelta(days=27)

train_df = df[df['Ride_start_datetime'] < cutoff_date].copy()
test_df = df[df['Ride_start_datetime'] >= cutoff_date].copy()

print(f"Training Set: {len(train_df):,} records ({train_df['Ride_start_datetime'].min()} to {train_df['Ride_start_datetime'].max()})")
print(f"Test Set: {len(test_df):,} records ({test_df['Ride_start_datetime'].min()} to {test_df['Ride_start_datetime'].max()})")
print(f"Split: {len(train_df)/len(df)*100:.1f}% train / {len(test_df)/len(df)*100:.1f}% test\n")

Training Set: 497,989 records (2017-10-01 16:15:00 to 2018-03-04 23:30:00)
Test Set: 89,785 records (2018-03-04 23:45:00 to 2018-03-31 23:45:00)
Split: 84.7% train / 15.3% test



In [8]:
cat_cols = ['Boarding_stop_stn', 'Alighting_stop_stn']

# All numerical features
num_cols = [
    'hour', 'minute', 'day', 'dayofweek', 'month', 'year', 'week_of_year',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 
    'dow_sin', 'dow_cos', 'month_sin', 'month_cos',
    'is_weekend', 'is_holiday', 'is_peak_hour']

# Add lag and rolling features
lag_roll_cols = [col for col in train_df.columns if col.startswith(('lag_', 'rolling_'))]
num_cols.extend(lag_roll_cols)

# Prepare feature matrices
X_train = train_df[cat_cols + num_cols].copy()
y_train = train_df['Passenger_Count'].copy()
X_test = test_df[cat_cols + num_cols].copy()
y_test = test_df['Passenger_Count'].copy()

print(f"Feature matrices:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  Total features: {len(cat_cols + num_cols)}\n")

Feature matrices:
  X_train: (497989, 35)
  X_test: (89785, 35)
  Total features: 35



In [9]:
# Helper functions
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if np.sum(mask) == 0:
        return 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = np.abs(y_true) + np.abs(y_pred)
    mask = denom != 0
    if np.sum(mask) == 0:
        return 0.0
    return np.mean(2.0 * np.abs(y_true[mask] - y_pred[mask]) / denom[mask]) * 100

def evaluate_model(y_true, y_pred, set_name="Set"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)
    print(f"\n{set_name} Performance:")
    print(f"  MAE:   {mae:.4f}")
    print(f"  RMSE:  {rmse:.4f}")
    print(f"  R²:    {r2:.4f}")
    print(f"  MAPE:  {mape:.2f}%")
    print(f"  sMAPE: {smape:.2f}%")
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape, 'sMAPE': smape}

def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test,
                             cat_cols=None, preprocessor=True):
    
    print(f"\n{'='*20} Training {model_name} {'='*20}\n")
    
    if model_name.lower() == 'catboost':
        all_features = list(X_train.columns)
        cat_feature_indices = [all_features.index(c) for c in cat_cols] if cat_cols else []
        train_pool = Pool(X_train, y_train, cat_features=cat_feature_indices)
        test_pool = Pool(X_test, y_test, cat_features=cat_feature_indices)
        model.fit(train_pool, eval_set=test_pool, use_best_model=False, verbose=100)
        y_train_pred = model.predict(train_pool)
        y_test_pred = model.predict(test_pool)
    else:
        if preprocessor:
            pipe = Pipeline([
                ('preprocessor', ColumnTransformer([
                    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols),
                    ('scaler', StandardScaler(), [c for c in X_train.columns if c not in cat_cols])
                ], remainder='drop', verbose_feature_names_out=False)),
                ('model', model)
            ])
            pipe.fit(X_train, y_train)
            y_train_pred = pipe.predict(X_train)
            y_test_pred = pipe.predict(X_test)
        else:
            model.fit(X_train, y_train)
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
    
    print("\nTraining Set Metrics:")
    train_metrics = evaluate_model(y_train, y_train_pred, "Training")
    
    print("\nTest Set Metrics:")
    test_metrics = evaluate_model(y_test, y_test_pred, "Test")
    
    # Overfitting check
    r2_gap = train_metrics['R2'] - test_metrics['R2']
    print(f"\nOverfitting R² Gap: {r2_gap:.4f}")
    if r2_gap > 0.15:
        print("⚠️ Significant overfitting")
    elif r2_gap > 0.05:
        print("⚡ Slight overfitting")
    else:
        print("✓ Good generalization")
    
    return y_train_pred, y_test_pred, train_metrics, test_metrics

In [11]:
xgb_model = XGBRegressor(
    n_estimators=500, 
    learning_rate=0.01, 
    max_depth=10, 
    subsample=0.8,
    colsample_bytree=0.8, 
    objective='reg:squarederror', 
    n_jobs=-1
)

y_train_pred_xgb, y_test_pred_xgb, train_metrics_xgb, test_metrics_xgb = train_and_evaluate_model("XGBoost", xgb_model, X_train, y_train, X_test, y_test, cat_cols=cat_cols)


==================== Training XGBoost ====================


Training Set Metrics:

Training Performance:
  MAE:   0.6311
  RMSE:  0.9507
  R²:    0.4027
  MAPE:  40.57%
  sMAPE: 36.68%

Test Set Metrics:

Test Performance:
  MAE:   0.6588
  RMSE:  1.0473
  R²:    0.2929
  MAPE:  42.23%
  sMAPE: 37.56%

Overfitting R² Gap: 0.1098
⚡ Slight overfitting


In [12]:
lgb_model = LGBMRegressor(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.05,
    num_leaves=40,
    reg_alpha=0.5,
    reg_lambda=1.0,
    n_jobs=-1
)

y_train_pred_lgb, y_test_pred_lgb, train_metrics_lgb, test_metrics_lgb = train_and_evaluate_model("LightGBM", lgb_model, X_train, y_train, X_test, y_test, cat_cols=cat_cols)


==================== Training LightGBM ====================

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009380 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1921
[LightGBM] [Info] Number of data points in the train set: 497989, number of used features: 97
[LightGBM] [Info] Start training from score 1.560767
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



Training Set Metrics:

Training Performance:
  MAE:   0.6483
  RMSE:  1.0064
  R²:    0.3306
  MAPE:  41.38%
  sMAPE: 37.12%

Test Set Metrics:

Test Performance:
  MAE:   0.6540
  RMSE:  1.0474
  R²:    0.2928
  MAPE:  41.73%
  sMAPE: 37.17%

Overfitting R² Gap: 0.0379
✓ Good generalization


In [13]:
cat_model = CatBoostRegressor(
    iterations=2000, 
    learning_rate=0.05, 
    depth=8, 
    l2_leaf_reg=3,
    loss_function='MAE', 
    eval_metric='MAE', 
    random_seed=42,
    task_type='CPU', 
    verbose=100
)

y_train_pred_cat, y_test_pred_cat, train_metrics_cat, test_metrics_cat = train_and_evaluate_model("CatBoost", cat_model, X_train, y_train, X_test, y_test, cat_cols=cat_cols, preprocessor=False)


==================== Training CatBoost ====================

0:	learn: 0.5603437	test: 0.5515743	best: 0.5515743 (0)	total: 182ms	remaining: 6m 4s
100:	learn: 0.5405859	test: 0.5298065	best: 0.5298065 (100)	total: 7.99s	remaining: 2m 30s
200:	learn: 0.5390099	test: 0.5283935	best: 0.5283933 (199)	total: 14.5s	remaining: 2m 10s
300:	learn: 0.5365236	test: 0.5260820	best: 0.5260820 (300)	total: 21s	remaining: 1m 58s
400:	learn: 0.5349953	test: 0.5248225	best: 0.5248225 (400)	total: 27.3s	remaining: 1m 48s
500:	learn: 0.5341115	test: 0.5242609	best: 0.5242609 (500)	total: 33.1s	remaining: 1m 39s
600:	learn: 0.5335405	test: 0.5237781	best: 0.5237781 (600)	total: 39.1s	remaining: 1m 31s
700:	learn: 0.5332227	test: 0.5236044	best: 0.5236044 (700)	total: 45s	remaining: 1m 23s
800:	learn: 0.5327494	test: 0.5233088	best: 0.5233088 (800)	total: 51.1s	remaining: 1m 16s
900:	learn: 0.5321435	test: 0.5228892	best: 0.5228892 (900)	total: 57.7s	remaining: 1m 10s
1000:	learn: 0.5316140	test: 0.522576